In [1]:
# ruff: noqa
import argparse
import logging
import os
from os.path import join
import sys
import warnings

from spatialdata import read_zarr

sys.path.insert(1, "/dss/dsshome1/0C/ra98gaq/Git/cellseg-benchmark")

from cellseg_benchmark import sdata_utils as su

/home/ubuntu/miniforge3/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/ubuntu/miniforge3/lib/python3.12/site-packages/anndata/__init__.py:44: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


In [2]:
#warnings.filterwarnings("ignore")

In [3]:
logger = logging.getLogger("shape_mapping")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s"))
logger.addHandler(handler)

In [4]:
def get_args(test_args=None):  # noqa: D103
    p = argparse.ArgumentParser(
    description="Creates a master sdata for a given sample, containing multiple segmentation results."
    )
    p.add_argument("sample", help="Sample name.")
    p.add_argument(
        "data_path",
        help="Path to folder with merscope output data (e.g. /cohort1/slide2/region0).",
    )
    p.add_argument(
        "zmode", choices=["z3"], help="Mode of master sdata. Either 'z3' or '3d' (currently only z3 is implemented)."
    )
    p.add_argument("data_dir", help="Output data folder.")
    p.add_argument(
        "--n_ficture",
        default=21,
        type=int,
        help="Consider Ficture model with n_ficture factors.",
    )
    p.add_argument("--run_date", type=str, help="run date (YYYYMMDD).", default=None)
    p.add_argument("--organism", type=str, help="organism.", default=None)
    p.add_argument("--slide", type=str, help="slide.", default=None)
    p.add_argument("--region", type=str, help="region.", default=None)
    p.add_argument("--cohort", type=str, help="cohort.", default=None)
    p.add_argument("--obs", action="append", default=[], metavar="KEY=VAL",
        help="Extra covariates to add to adata.obs (repeatable), e.g. --obs tissue=brain.")
    if test_args is not None:
        return p.parse_args(test_args)
    else:
        return p.parse_args()

In [5]:
# Simulate CLI arguments inside Jupyter
args = get_args(
    [
        "SynergyLung_s1_r0",
        "/dss/dssfs03/pn52re/pn52re-dss-0000/Synergy-projects-temp/MouseLung-Sijia/output-20250704_AGLiesz-Sijia-MouseLung-Slide1-Stroke_VMSC03901/region_R1-Stroke",
        "z3",
        "/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark",
        "--cohort",
        "SynergyLung",
        "--slide",
        "1",
        "--region",
        "0",
        "--organism",
        "mouse",
        "--run_date",
        "20250704",
        "--obs",
        "condition=stroke",
    ]
)

In [7]:
# Simulate CLI arguments inside Jupyter
args = get_args(
    [
        "aging_s11_r0",
        "/dss/dssfs03/pn52re/pn52re-dss-0000/202405-Htra1-and-Aging/merfish_output/20250508_Aging-Slide10-cp-WT308_3m-WT309_3m-WT132_24m/region_0-WT132_24m",
        "z3",
        "/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark",
        "--cohort",
        "aging",
        "--slide",
        "10",
        "--region",
        "0",
        "--organism",
        "mouse",
        "--run_date",
        "20250508",
        "--obs",
        "genotype=WT",
        "--obs",
        "age_months=24",
        "--obs",
        "animal_id=132"
    ]
)

In [12]:
# Simulate CLI arguments inside Jupyter
args = get_args(
    [
        "htra1_s4_r1",
        "/dss/dssfs03/pn52re/pn52re-dss-0000/202405-Htra1-and-Aging/merfish_output/20240829_Htra1-Slide04-cp-TG69-KO305/region_1-TG69",
        "z3",
        "/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark",
        "--cohort",
        "htra1",
        "--slide",
        "4",
        "--region",
        "1",
        "--organism",
        "mouse",
        "--run_date",
        "20240829",
        "--obs",
        "genotype=TG",
        "--obs",
        "age_months=6",
        "--obs",
        "animal_id=69"
    ]
)

In [13]:
args

Namespace(sample='htra1_s4_r1', data_path='/dss/dssfs03/pn52re/pn52re-dss-0000/202405-Htra1-and-Aging/merfish_output/20240829_Htra1-Slide04-cp-TG69-KO305/region_1-TG69', zmode='z3', data_dir='/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark', n_ficture=21, run_date='20240829', organism='mouse', slide='4', region='1', cohort='htra1', obs=['genotype=TG', 'age_months=6', 'animal_id=69'])

In [14]:
NONES = {"", "None", "none", "null", "NULL", None}
for k in ["organism", "slide", "region", "cohort"]:
    if getattr(args, k) in NONES:
        setattr(args, k, None)

In [15]:
extra_obs = {}
for kv in args.obs:
    k, v = kv.split("=", 1)
    extra_obs[k] = None if v in NONES else v

In [16]:
extra_obs

{'genotype': 'TG', 'age_months': '6', 'animal_id': '69'}

In [17]:
#

In [18]:
%%time
logger.info("Importing images and points...")
su.process_merscope(args.sample, args.data_dir, args.data_path, zmode=args.zmode)

2025-10-24 10:16:44,679 [INFO]: Importing images and points...


Skipping htra1_s4_r1: z3 file already exists
CPU times: user 4.01 ms, sys: 552 μs, total: 4.56 ms
Wall time: 4.91 ms


In [19]:
sdata_path = join(args.data_dir, "samples", args.sample)
sdata_main = read_zarr(join(sdata_path, "sdata_z3.zarr"))

version mismatch: detected: RasterFormatV02, requested: FormatV04
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/ubuntu/miniforge3/lib/python3.12/site-packages/zarr/creation.py:610: UserWarning: ignoring keyword argument 'read_only'
  compre

In [20]:
sdata_main

SpatialData object, with associated Zarr store: /dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark/samples/htra1_s4_r1/sdata_z3.zarr
├── Images
│     └── 'htra1_s4_r1_z3': DataTree[cyx] (7, 56200, 67420), (7, 28100, 33710), (7, 14050, 16855), (7, 7025, 8427), (7, 3512, 4213)
├── Points
│     └── 'htra1_s4_r1_transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
├── Shapes
│     └── 'boundaries_Cellpose_1_nuclei_model': GeoDataFrame shape: (52608, 1) (2D shapes)
└── Tables
      └── 'adata_Cellpose_1_nuclei_model': AnnData (52608, 500)
with coordinate systems:
    ▸ 'global', with elements:
        htra1_s4_r1_z3 (Images), htra1_s4_r1_transcripts (Points), boundaries_Cellpose_1_nuclei_model (Shapes)
    ▸ 'micron', with elements:
        htra1_s4_r1_z3 (Images), htra1_s4_r1_transcripts (Points), boundaries_Cellpose_1_nuclei_model (Shapes)
    ▸ 'pixel', with elements:
        htra1_s4_r1_z3 (Images), htra1_s4_r1_transcripts (Points), boundaries_Cellpose_1_nuclei_model (Shape

In [ ]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs

In [ ]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs.columns

In [ ]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs["age_months"]

In [ ]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs["genotype"]

In [ ]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs["animal_id"]

In [21]:
logger.info("Integrating segmentation data from available methods into main sdata...")
# only work on methods with actual data available
seg_methods = [
    method
    for method in os.listdir(join(sdata_path, "results"))
    if os.path.isdir(join(sdata_path, "results", method, "sdata.zarr"))
]

2025-10-24 10:17:33,596 [INFO]: Integrating segmentation data from available methods into main sdata...


In [22]:
seg_methods

['Cellpose_1_nuclei_model',
 'Proseg_Cellpose_1_nuclei_model',
 'Baysor_2D_Cellpose_1_nuclei_model_1.0']

In [23]:
extra_obs

{'genotype': 'TG', 'age_months': '6', 'animal_id': '69'}

In [24]:
su.integrate_segmentation_data(
    sdata_path,
    seg_methods,
    sdata_main,
    run_date=args.run_date,
    organism=args.organism,
    slide=args.slide,
    region=args.region,
    cohort=args.cohort,
    write_to_disk=True,
    data_path=args.data_path,
    logger=logger,
    **extra_obs,
)

  0%|          | 0/3 [00:00<?, ?it/s]2025-10-24 10:17:43,932 [INFO]: Adding Cellpose_1_nuclei_model...
2025-10-24 10:17:43,938 [WARNING]: Skipping boundary import of Cellpose_1_nuclei_model as boundaries_Cellpose_1_nuclei_model exist already.
2025-10-24 10:17:43,939 [WARNING]: Skipping adata import of Cellpose_1_nuclei_model as adata_Cellpose_1_nuclei_model exist already.
2025-10-24 10:17:43,940 [INFO]: Adding Proseg_Cellpose_1_nuclei_model...
2025-10-24 10:17:45,841 [INFO]: Adding shapes of Proseg_Cellpose_1_nuclei_model...
 33%|███▎      | 1/3 [00:01<00:03,  1.91s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/dss/dssfs03/pn52re/pn52re-dss-0001/cellseg-benchmark/samples/htra1_s4_r1/results/Proseg_Cellpose_1_nuclei_model/sdata.zarr/.sopa_cache/transcript_patches/0/cell-polygons-layers.geojson.gz'

In [21]:
#

In [22]:
#################

In [20]:
# test sdata

In [25]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs.columns

Index(['region', 'slide', 'spt_region', 'cell_type_incl_low_quality_revised',
       'cell_type_mmc_incl_low_quality_clusters',
       'cell_type_mmc_incl_low_quality', 'cell_type_incl_mixed_revised',
       'cell_type_mmc_incl_mixed_clusters', 'cell_type_mmc_incl_mixed',
       'cell_type_revised', 'cell_type_mmc_raw_clusters', 'cell_type_mmc_raw',
       'cell_id', 'area', 'volume_sum', 'volume_final', 'num_z_planes',
       'size_normalized', 'surface_to_volume_ratio', 'sphericity', 'solidity',
       'elongation', 'genotype', 'age_months', 'run_date', 'organism',
       'cohort', 'sample', 'animal_id', 'condition'],
      dtype='object')

In [26]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs["age_months"]

aaaaaaaa-1    24
aaaaaaab-1    24
aaaaaaac-1    24
aaaaaaad-1    24
aaaaaaae-1    24
              ..
aaaaildm-1    24
aaaaildn-1    24
aaaaildo-1    24
aaaaildp-1    24
aaaailea-1    24
Name: age_months, Length: 35649, dtype: object

In [23]:
sdata_main.tables["adata_Cellpose_1_nuclei_model"].obs["condition"]

aaaaaaaa-1    WT_24
aaaaaaab-1    WT_24
aaaaaaac-1    WT_24
aaaaaaad-1    WT_24
aaaaaaae-1    WT_24
              ...  
aaaaildm-1    WT_24
aaaaildn-1    WT_24
aaaaildo-1    WT_24
aaaaildp-1    WT_24
aaaailea-1    WT_24
Name: condition, Length: 35649, dtype: object

In [24]:
sdata_main.tables["adata_Cellpose_1_Merlin"].obs.columns

Index(['fov', 'volume', 'center_x', 'center_y', 'min_x', 'min_y', 'max_x',
       'max_y', 'anisotropy', 'transcript_count', 'perimeter_area_ratio',
       'Fth1_raw', 'Fth1_high_pass', 'DAPI_raw', 'DAPI_high_pass', 'App_raw',
       'App_high_pass', 'Aldoc_raw', 'Aldoc_high_pass', 'Sst_raw',
       'Sst_high_pass', 'Plp1_raw', 'Plp1_high_pass', 'PolyT_raw',
       'PolyT_high_pass', 'region', 'slide', 'dataset_id', 'cells_region',
       'spt_region', 'cell_type_incl_low_quality_revised',
       'cell_type_mmc_incl_low_quality_clusters',
       'cell_type_mmc_incl_low_quality', 'cell_type_incl_mixed_revised',
       'cell_type_mmc_incl_mixed_clusters', 'cell_type_mmc_incl_mixed',
       'cell_type_revised', 'cell_type_mmc_raw_clusters', 'cell_type_mmc_raw',
       'cell_id', 'area', 'volume_sum', 'volume_final', 'num_z_planes',
       'size_normalized', 'surface_to_volume_ratio', 'sphericity', 'solidity',
       'elongation', 'animal_id', 'run_date', 'organism', 'cohort', 'sample',
  

In [57]:
sdata_main.tables["adata_Cellpose_1_Merlin"].obs

,fov,volume,center_x,center_y,min_x,min_y,max_x,max_y,anisotropy,transcript_count,...,size_normalized,surface_to_volume_ratio,sphericity,solidity,elongation,condition,run_date,organism,cohort,sample
EntityID,,,,,,,,,,,,,,,,,,,,,
3032071400013100126,840,2059.148908,7216.183057,479.060704,7207.846355,470.933895,7224.305598,486.959026,1.031700,3,...,14.003907,0.256930,0.970690,0.994144,0.032101,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400014100180,918,2139.125086,7504.486833,498.349751,7496.258106,489.906420,7512.488711,506.379802,1.009113,3,...,14.273269,0.250883,0.979986,1.000000,0.070396,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400016100184,1155,1993.494412,8556.587687,298.372080,8549.290734,290.198104,8565.302442,306.362748,1.135827,569,...,13.778846,0.273867,0.882477,0.990295,0.042211,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400016100195,1155,1507.780768,8632.307395,307.804110,8625.148066,300.586582,8638.940023,314.807612,1.084933,1,...,11.983245,0.309654,0.912657,0.983984,0.109678,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400016100200,1155,1870.495178,8620.070741,312.002451,8611.488222,304.131019,8628.272423,319.113140,1.213755,124,...,13.347000,0.277245,0.917727,0.980912,0.163323,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032071400416100177,1116,2344.589468,8437.787779,8130.221066,8428.794190,8121.712069,8446.369621,8138.413806,1.076666,3,...,14.943033,0.243381,0.950079,0.987811,0.033608,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400416100183,1116,1980.095639,8309.034979,8145.903951,8301.228004,8137.594659,8316.478445,8154.072963,1.057263,5,...,13.732463,0.262205,0.969239,0.995752,0.046148,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
3032071400416100184,1116,1502.556981,8424.599640,8144.706371,8418.229115,8137.784821,8431.230806,8151.444653,1.068816,3,...,11.962469,0.303448,0.953675,0.992073,0.044607,control,20250718,mouse,SynergyLung,SynergyLung_s2_r0
